6)	Download the breast cancer Wisconsin dataset from the UCI Machine learning repository: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/ -- specifically, use the WDBC dataset.



a. 	Fit a univariate OLS linear regression model to predict whether a given sample is benign or malignant using the Standard Error of the area as the independent variable.  Specifically, using 0.5 as your cutoff criterion, “classify” samples in the test set by whether they are predicted to be malignant. Using 10-fold cross validation, what is the accuracy of this approach?


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import statsmodels as sm
import os
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/My Drive/EMSE6575/wdbc.data"
df = pd.read_csv(path)

In [5]:
X, y = df.drop(df.columns[:2], axis=1), df[df.columns[1]]
y.replace({'M':1, 'B':0}, inplace=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=1)

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 494 to 283
Data columns (total 30 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   17.99     482 non-null    float64
 1   10.38     482 non-null    float64
 2   122.8     482 non-null    float64
 3   1001      482 non-null    float64
 4   0.1184    482 non-null    float64
 5   0.2776    482 non-null    float64
 6   0.3001    482 non-null    float64
 7   0.1471    482 non-null    float64
 8   0.2419    482 non-null    float64
 9   0.07871   482 non-null    float64
 10  1.095     482 non-null    float64
 11  0.9053    482 non-null    float64
 12  8.589     482 non-null    float64
 13  153.4     482 non-null    float64
 14  0.006399  482 non-null    float64
 15  0.04904   482 non-null    float64
 16  0.05373   482 non-null    float64
 17  0.01587   482 non-null    float64
 18  0.03003   482 non-null    float64
 19  0.006193  482 non-null    float64
 20  25.38     482 non-null    floa

In [13]:
from statsmodels.regression.linear_model import OLS

In [15]:
res = OLS(y_train,X_train.iloc[:, [16]]).fit()
# Get Coefficients
coeffs_ols = res.params
print(f'coefficients : {coeffs_ols}')

# Prediction
ols_preds= res.predict()

# Model summary
res.summary()

coefficients : 0.05373    7.611696
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      M   R-squared (uncentered):                   0.307
Model:                            OLS   Adj. R-squared (uncentered):              0.305
Method:                 Least Squares   F-statistic:                              213.0
Date:                Fri, 31 Mar 2023   Prob (F-statistic):                    3.39e-40
Time:                        09:25:41   Log-Likelihood:                         -356.87
No. Observations:                 482   AIC:                                      715.7
Df Residuals:                     481   BIC:                                      719.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0.05373        7.6117      0.522     14.593      0.000       6.587       8.637
==============================================================================
Omnibus:                       62.085   Durbin-Watson:                   1.929
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              233.044
Skew:                          -0.520   Prob(JB):                     2.48e-51
Kurtosis:                       6.244   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [28]:
kf = KFold(n_splits=10)
accuracy = 0
for train, test in kf.split(X_train.iloc[:, [16]]):
    res_t = OLS(y_train.iloc[train],X_train.iloc[:, [16]].iloc[train]).fit()
    test_res = res_t.predict(X_train.iloc[:, [16]].iloc[test]).apply(lambda x: 1 if x>=0.5 else 0)
    accuracy += accuracy_score(y_train.iloc[test], test_res)

10 fold cross validation accuracy is  0.6160289115646258


In [29]:
print('10 fold cross validation accuracy: ', accuracy/10)

10 fold cross validation accuracy:  0.6160289115646258


b. 	Does linear regression perform well at this classification task? How does it compare to a model that consistently estimates “benign”? How does it compare to the logistic regression model calculated in #1?

In [22]:
actual_test_res = res.predict(X_test.iloc[:, [16]]).apply(lambda x: 1 if x>=0.5 else 0)

In [21]:
print('Accuracy of test set: ', accuracy_score(y_test, actual_test_res))
print('Accuracy of the entire model: ', accuracy_score(y_test, [0]*len(y_test)))

Accuracy of test set:  0.7093023255813954
Accuracy of the entire model:  0.627906976744186


c. 	Explain the difference in performance between the logistic and linear regression models.

In [25]:
lr = LogisticRegression(random_state=1)
lr.fit(X_train.iloc[:, [16]], y_train)
y_pred = lr.predict(X_test.iloc[:, [16]])

In [26]:
print('Accuracy of logistic regression of test set: ', accuracy_score(y_test, y_pred))

Accuracy of logistic regression of test set:  0.627906976744186
